# Peakfinder

## Introduction

This notebook takes input data from ASR-recordings in the form of an input folder containing the CSV files for each recording day and animal, giving output files regarding the average reaction of each animal to each experiment. <br>The code is annotated and a more extensive explanation is contained within `Protocol.pdf`. 

## Imports

If this is your first time running this, you may need to install various python packages using pip. <br>
Open the terminal, either the windows command console or the VSC terminal, and run the command `pip install MODULENAME` for every module listed in the cell below (for example, `pip install pandas` to install pandas). Some of them may already be installed. Repeat this for the imports in `Analyzer.ipynb`.

In [1]:
import pandas as pd       
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import defaultdict, Counter
from IPython.display import clear_output
import re
from scipy.optimize import curve_fit
from scipy.integrate import simpson

## Configuration

In [2]:
input_dir = "Peakfinder Input" # - - - - - - - - - - - - - - - - - - - - - - - - - - folder containing folders with .CSV files; change this to your own input directory

output_dir = "[1] Peakfinder Results" # - - - - - - - - - - - - - - - - - - - - - - - - folder containing final results

export_folder = r"C:\Users\Galahad\Desktop\Thesis\images"

verbose = True # - - - - - - - - - - - - - - - - - - - - - - - - - - - - enables more output messages which can help identify problems; the notebook will only return errors if False

discard_peakless = False # - - - - - - - - - - - - - - - - - - - - - - -  discard recordings without peaks from analysis

include_decay_fit = False # - - - - - - - - - - - - - - - - - - - - - -  include regular exponential decay in the output

output = False # - - - - - - - - - - - - - - - - - - - - - - - - - - - -  output results; set to False for testing if you do not want any change in output files when running the notebook

detection_threshold = 24 # - - - - - - - - - - - - - - - - - - - - - - - threshold multiplier for detecting peaks, ValueG has to exceed k times the background noise per animal

RT_threshold = 12 # - - - - - - - - - - - - - - - - - - - - - - - - - - threshold multiplier for measuring reaction time, ValueG has to exceed k times the background noise per animal

length = 500 # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - length of the time window in milliseconds (both before and after the startle)

example_number = 33 # - - - - - - - - - - - - - - - - - - - - - - - - -  index of the trial used for the example plot

background_offset = 200 # - - - - - - - - - - - - - - - - - - - - - - -  measure background noise up to this many ms before the startle time (increase if a prepulse is played this or further away from the startle)

detection_offset = 100 # - - - - - - - - - - - - - - - - - - - - - - - - check for signal up to this many ms after the startle time (for detection)

RMS_window = 200 # - - - - - - - - - - - - - - - - - - - - - - - check for signal up to this many ms after the startle time (for measurement)

peak_window = 100 

PTTA_window = 200

PTPA_window = 50

decay_fit_window = 500

first_amount = 10

mode = ""#"control"

filenames = ["OVERVIEW", "RESULTS", f"RESULTS_T{first_amount}", f"RESULTS_MT{first_amount}"] # - - - result file names (overview, main results, top 10 results, results minus the top 10)


#exclude = ["offset_PPI", "offset_ASR"] # - - - - - - - - - - - - - - -  experiment types to be excluded from analysis
#exclude = ["ASR_control", "gap_depth"]
exclude = None

In [3]:
# Input startle times for every experiment
startle_times = {

    "ASR_control":      10000,
    "gap_depth":        10100,
    "gap_duration_10":  10060,
    "gap_duration_20":  10070,
    "gap_duration_4":   10054,
    "gap_duration_50":  10100,
    "gap_duration_8":   10058,
    "offset_ASR":       10010,
    "offset_PPI_10":    10010,
    "offset_PPI_12":    10012,
    "offset_PPI_14":    10014,
    "offset_PPI_16":    10016,
    "offset_PPI_18":    10018,
    "offset_PPI_20":    10020,
    "offset_PPI_4":     10004,
    "offset_PPI_6":     10006,
    "offset_PPI_8":     10008,
    "offset_PPI_50":    10050,
    "tone_in_noise":    10100
    
}

In [4]:
# interpulse intervals
IPI = {

    "ASR_control":      None,
    "gap_depth":        50,
    "gap_duration_10":  10,
    "gap_duration_20":  20,
    "gap_duration_4":   4,
    "gap_duration_50":  50,
    "gap_duration_8":   8,
    "offset_ASR":       None,
    "offset_PPI_10":    10,
    "offset_PPI_12":    12,
    "offset_PPI_14":    14,
    "offset_PPI_16":    16,
    "offset_PPI_18":    18,
    "offset_PPI_20":    20,
    "offset_PPI_4":     4,
    "offset_PPI_6":     6,
    "offset_PPI_8":     8,
    "offset_PPI_50":    50,
    "tone_in_noise":    50

}

In [5]:
# ...existing code...
# Calculate prepulse times for each experiment
prepulse_times = {}

for exp_name in startle_times:
    startle_time = startle_times[exp_name]
    ipi = IPI.get(exp_name)
    if startle_time is None or ipi is None:
        prepulse_times[exp_name] = None
    else:
        prepulse_times[exp_name] = startle_time - ipi

if verbose:
    print("Prepulse times (startle_time - IPI):")
    for exp, ppt in prepulse_times.items():
        print(f"{exp}: {ppt}")
# ...existing code...

Prepulse times (startle_time - IPI):
ASR_control: None
gap_depth: 10050
gap_duration_10: 10050
gap_duration_20: 10050
gap_duration_4: 10050
gap_duration_50: 10050
gap_duration_8: 10050
offset_ASR: None
offset_PPI_10: 10000
offset_PPI_12: 10000
offset_PPI_14: 10000
offset_PPI_16: 10000
offset_PPI_18: 10000
offset_PPI_20: 10000
offset_PPI_4: 10000
offset_PPI_6: 10000
offset_PPI_8: 10000
offset_PPI_50: 10000
tone_in_noise: 10050


In [6]:
# Input sex for every animal
sex_dict = {
    "1":    "female",
    "2":    "male",
    "4":    "male",
    "6":    "female",
    "10":   "female",
    "15":   "male"
}

In [7]:
# CSV file settings (insert columns like how they'd appear in an excel file, as in starting with column 1)

data_start =           5   # row where recording data starts

animal_column =        2   # column "Animal. No."
experiment_column =    8   # column "Exp. No."
weight_column =        6   # column "Weight"
number_column =        18  # column "TrialNo"
name_column =          17  # column "Trial"
time_column =          19  # column "MS"
value_column =         20  # column "ValueG"

# adjust columns to be 0-indexed
experiment_column, weight_column, number_column, name_column, time_column, value_column = (x - 1 for x in (experiment_column, weight_column, number_column, name_column, time_column, value_column))

## Slicing

In [8]:
segments = {}
startles = {}

for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.lower().endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path, low_memory=False, skiprows=[0, 2])

            animal_number = df.iloc[data_start + 1, animal_column]
            date = file[:-4].split('_')[1]
            animal_str = f"Animal{animal_number}"

            df.iloc[:, value_column] = pd.to_numeric(df.iloc[:, value_column], errors='coerce')

            df.iloc[:, value_column] = abs(df.iloc[:, value_column])

            df['trial'] = df.iloc[:, name_column]
            df['trial_shift'] = df['trial'].shift(1)
            df['new_segment'] = df['trial'] != df['trial_shift']
            df['segment_id'] = df['new_segment'].cumsum()

            repetition = {}

            for segment_id, segment_df in df.groupby('segment_id'):
                trial_name = segment_df.iloc[0]['trial']
                repetition[trial_name] = repetition.get(trial_name, 0) + 1

                segment_startle = startle_times.get(trial_name)
                if segment_startle is None:
                    raise Exception(f"Missing startle time for {trial_name}. Please make sure it is included in the startle_times dictionary.")

                segment_name = f"{animal_str}_{date}_{trial_name}_Rep{repetition[trial_name]}"

                segment_length = len(segment_df)
                time_col = np.arange(0, segment_length * 2, 2)
                segment_data = pd.DataFrame({
                    0: time_col,
                    1: segment_df.iloc[:, value_column].values
                })

                segments[segment_name] = segment_data
                startles[segment_name] = segment_startle

counts = Counter([segment_df.iloc[0]['trial'] for _, segment_df in df.groupby('segment_id')])

clear_output(wait=False)

if verbose: 
    print(f"Processed {len(segments)} segments.") # 1710 expected
    print("Repetition count per experiment:")
    for name, count in sorted(counts.items()):
        print(f"{name}: {count}") # 5 each expected

Processed 1645 segments.
Repetition count per experiment:
ASR_control: 5
gap_depth: 5
gap_duration_10: 5
gap_duration_20: 5
gap_duration_4: 5
gap_duration_50: 5
gap_duration_8: 5
offset_ASR: 5
offset_PPI_10: 5
offset_PPI_12: 5
offset_PPI_14: 5
offset_PPI_16: 5
offset_PPI_18: 3
offset_PPI_20: 4
offset_PPI_4: 5
offset_PPI_50: 4
offset_PPI_6: 4
offset_PPI_8: 5
tone_in_noise: 5


In [9]:
# Get background noise per animal
animal_noise_results = {}

experiment_keys = [t[0] for t in sorted(counts.items())]

for key in segments.keys():
    # Extract animal name
    animal = key.split('_')[0]

    # Find experiment prefix in key
    matching_prefix = None
    for exp_key in experiment_keys:
        if exp_key in key:
            matching_prefix = exp_key
            break

    if matching_prefix is None:
        print(f"No experiment prefix found for key: {key}")
        continue

    startle_time = startle_times.get(matching_prefix)
    if startle_time is None:
        print(f"No startle time found for experiment: {matching_prefix}")
        continue

    df = segments[key]

    # Convert time column to numeric
    df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')

    start_time = 0
    end_time = startle_time - background_offset  # measure up to 200ms before the startle
    if end_time <= start_time:
        print(f"Invalid time window for {key}: startle_time={startle_time}")
        continue

    mask = (df.iloc[:, 0] >= start_time) & (df.iloc[:, 0] < end_time)
    background_window = df.loc[mask].iloc[:, 1]

    if background_window.empty:
        print(f"No data in background window for {key}")
        continue

    noise_level = background_window.std()

    # Store noise levels per animal and experiment across repetitions
    animal_noise_results.setdefault(animal, {}).setdefault(matching_prefix, []).append(noise_level)

# Median noise across repetitions per experiment per animal
animal_noise_median = {}
for animal, exp_dict in animal_noise_results.items():
    animal_noise_median[animal] = {}
    for exp, noise_list in exp_dict.items():
        animal_noise_median[animal][exp] = float(np.median(noise_list))

# Median noise across experiments per animal
animal_noise = {}
for animal, exp_dict in animal_noise_median.items():
    all_exp_values = list(exp_dict.values())
    if all_exp_values:
        animal_noise[animal] = round(float(np.median(all_exp_values)), 2)
    else:
        animal_noise[animal] = None

if verbose:
    print(f"Collected background noise for {len(animal_noise)} animals:")
    print(animal_noise)

""" 
# ...existing code...

# Mean noise across repetitions per experiment per animal
animal_noise_mean = {}
for animal, exp_dict in animal_noise_results.items():
    animal_noise_mean[animal] = {}
    for exp, noise_list in exp_dict.items():
        animal_noise_mean[animal][exp] = float(np.mean(noise_list))

# Mean noise across experiments per animal
animal_noise = {}
for animal, exp_dict in animal_noise_mean.items():
    all_exp_values = list(exp_dict.values())
    if all_exp_values:
        animal_noise[animal] = round(float(np.mean(all_exp_values)), 2)
    else:
        animal_noise[animal] = None

if verbose:
    print(f"Collected background noise for {len(animal_noise)} animals (mean):")
    print(animal_noise)
# ...existing code... """

Collected background noise for 6 animals:
{'Animal10': 0.27, 'Animal15': 0.62, 'Animal1': 0.34, 'Animal2': 0.3, 'Animal4': 0.4, 'Animal6': 0.27}


' \n# ...existing code...\n\n# Mean noise across repetitions per experiment per animal\nanimal_noise_mean = {}\nfor animal, exp_dict in animal_noise_results.items():\n    animal_noise_mean[animal] = {}\n    for exp, noise_list in exp_dict.items():\n        animal_noise_mean[animal][exp] = float(np.mean(noise_list))\n\n# Mean noise across experiments per animal\nanimal_noise = {}\nfor animal, exp_dict in animal_noise_mean.items():\n    all_exp_values = list(exp_dict.values())\n    if all_exp_values:\n        animal_noise[animal] = round(float(np.mean(all_exp_values)), 2)\n    else:\n        animal_noise[animal] = None\n\nif verbose:\n    print(f"Collected background noise for {len(animal_noise)} animals (mean):")\n    print(animal_noise)\n# ...existing code... '

In [10]:
# Cut the segments
adjusted_segments = {}
for entry in segments:
    df = segments[entry]
    reference_time = startles[entry]  

    # Filter the data to get rows within the 1000 ms before and after the reference time
    start_time = reference_time - length
    end_time = reference_time + length

    # Filter the dataframe to include only the data within the time range
    filtered_df = df[(df.iloc[:, 0] >= start_time) & (df.iloc[:, 0] <= end_time)].copy()

    # Adjust the time column to set the reference time to 0
    filtered_df.iloc[:, 0] = filtered_df.iloc[:, 0] - reference_time

    filtered_df.reset_index(drop=True, inplace=True) 

    adjusted_segments[entry] = filtered_df

if verbose: print(f"Adjusted {len(adjusted_segments)} segments.")

Adjusted 1645 segments.


In [11]:
# Record the order of all segments and save as an Excel file, with a row per animal and date (DATE as a column)
# Instead of segment name, save the experiment name

segment_order = defaultdict(list)

for i, key in enumerate(adjusted_segments.keys()):
    parts = key.split('_')
    animal = parts[0]
    date = parts[1]
    # Extract experiment name (everything after animal and date, before Rep)
    experiment_parts = []
    for p in parts[2:]:
        if p.lower().startswith("rep"):
            break
        experiment_parts.append(p)
    experiment_name = "_".join(experiment_parts)
    segment_order[(animal, date)].append((i + 1, experiment_name))  # 1-based index

order_rows = []
for (animal, date), segments in segment_order.items():
    for idx, experiment in segments:
        row = {
            'animal': animal,
            'date': date,
            'order_index': idx,
            'experiment': experiment
        }
        order_rows.append(row)

order_df = pd.DataFrame(order_rows)
if output:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    order_df.to_excel(os.path.join(output_dir, "SegmentOrder.xlsx"), index=False)

if verbose:
    print("Saved segment order to:", os.path.join(output_dir, "SegmentOrder.xlsx"))

Saved segment order to: [1] Peakfinder Results\SegmentOrder.xlsx


In [12]:
# Find top 10 recordings

import random
animal_date_keys = defaultdict(list)
minus_top_10_keys = list(adjusted_segments.keys())

for key in adjusted_segments.keys():
    parts = key.split('_')
    animal = parts[0]
    date = parts[1]
    animal_date_keys[(animal, date)].append(key)

# Take first 10 keys per animal per date
top_10_keys = []
for (animal, date), keys in animal_date_keys.items():

    start_idx = random.randint(0, len(keys) - first_amount)
    first_10_keys = keys[start_idx:start_idx + first_amount] if mode == "control" else keys[:first_amount]

    top_10_keys.extend(first_10_keys)

# Remove top 10 entries per animal and day from minus top 10 keyset
for key in minus_top_10_keys: 
    if key in top_10_keys: 
        minus_top_10_keys.remove(key)

if verbose: print(f"Found {len(top_10_keys)} segments for the first ten recordings.")

Found 180 segments for the first ten recordings.


In [13]:
print(top_10_keys)
print(len(minus_top_10_keys))
print(len(adjusted_segments))

['Animal10_April16_gap_duration_4_Rep1', 'Animal10_April16_gap_depth_Rep1', 'Animal10_April16_offset_PPI_4_Rep1', 'Animal10_April16_offset_PPI_50_Rep1', 'Animal10_April16_offset_PPI_8_Rep1', 'Animal10_April16_offset_PPI_14_Rep1', 'Animal10_April16_offset_PPI_6_Rep1', 'Animal10_April16_offset_ASR_Rep1', 'Animal10_April16_gap_duration_10_Rep1', 'Animal10_April16_offset_PPI_12_Rep1', 'Animal10_June26_offset_PPI_14_Rep1', 'Animal10_June26_offset_PPI_50_Rep1', 'Animal10_June26_offset_PPI_4_Rep1', 'Animal10_June26_gap_depth_Rep1', 'Animal10_June26_offset_PPI_16_Rep1', 'Animal10_June26_offset_PPI_18_Rep1', 'Animal10_June26_tone_in_noise_Rep1', 'Animal10_June26_gap_duration_10_Rep1', 'Animal10_June26_ASR_control_Rep1', 'Animal10_June26_gap_depth_Rep2', 'Animal10_May20_offset_PPI_4_Rep1', 'Animal10_May20_gap_duration_4_Rep1', 'Animal10_May20_offset_PPI_10_Rep1', 'Animal10_May20_offset_PPI_4_Rep2', 'Animal10_May20_gap_duration_20_Rep1', 'Animal10_May20_offset_PPI_20_Rep1', 'Animal10_May20_offset

In [14]:
if exclude is not None:
    before = len(adjusted_segments)
    # Remove excluded experiments
    for key in adjusted_segments.keys():
        adjusted_segments = {   # remove excluded keys
        key: value
        for key, value in adjusted_segments.items()
        if not any(word in key for word in exclude)
    }
    if verbose: print(f"Excluded {before-len(adjusted_segments)} segments.")

## Signal Detection

In [15]:
animal_thresholds = {animal: animal_noise[animal] * RT_threshold for animal in animal_noise}
animal_thresholds_det = {animal: animal_noise[animal] * detection_threshold for animal in animal_noise}

In [16]:
print(f"Animal thresholds for reaction time: {animal_thresholds}")
print(f"Animal thresholds for detection: {animal_thresholds_det}")

Animal thresholds for reaction time: {'Animal10': 3.24, 'Animal15': 7.4399999999999995, 'Animal1': 4.08, 'Animal2': 3.5999999999999996, 'Animal4': 4.800000000000001, 'Animal6': 3.24}
Animal thresholds for detection: {'Animal10': 6.48, 'Animal15': 14.879999999999999, 'Animal1': 8.16, 'Animal2': 7.199999999999999, 'Animal4': 9.600000000000001, 'Animal6': 6.48}


With mean: <br>
Animal thresholds for reaction time: {'Animal10': 3.06, 'Animal15': 8.64, 'Animal1': 3.4799999999999995, 'Animal2': 2.94, 'Animal4': 4.14, 'Animal6': 3.0} <br>
Animal thresholds for detection: {'Animal10': 12.24, 'Animal15': 34.56, 'Animal1': 13.919999999999998, 'Animal2': 11.76, 'Animal4': 16.56, 'Animal6': 12.0}

In [17]:
signal_results = {}

for key, df in adjusted_segments.items():
    animal = key.split('_')[0]
    noise = animal_noise.get(animal)

    # Set post-startle window
    mask = (df.iloc[:, 0] >= 0) & (df.iloc[:, 0] <= detection_offset)
    window = df.loc[mask].iloc[:, 1]

    # Threshold detection
    detected = not window.empty and window.max() > detection_threshold * noise
    signal_results[key] = detected

if verbose: print(signal_results)

{'Animal10_April16_gap_duration_4_Rep1': np.True_, 'Animal10_April16_gap_depth_Rep1': np.True_, 'Animal10_April16_offset_PPI_4_Rep1': np.True_, 'Animal10_April16_offset_PPI_50_Rep1': np.True_, 'Animal10_April16_offset_PPI_8_Rep1': np.True_, 'Animal10_April16_offset_PPI_14_Rep1': np.True_, 'Animal10_April16_offset_PPI_6_Rep1': np.True_, 'Animal10_April16_offset_ASR_Rep1': np.False_, 'Animal10_April16_gap_duration_10_Rep1': np.True_, 'Animal10_April16_offset_PPI_12_Rep1': np.True_, 'Animal10_April16_offset_ASR_Rep2': np.False_, 'Animal10_April16_offset_PPI_16_Rep1': np.True_, 'Animal10_April16_offset_PPI_4_Rep2': np.True_, 'Animal10_April16_gap_depth_Rep2': np.True_, 'Animal10_April16_offset_PPI_12_Rep2': np.True_, 'Animal10_April16_offset_PPI_10_Rep1': np.True_, 'Animal10_April16_offset_PPI_18_Rep1': np.True_, 'Animal10_April16_offset_PPI_4_Rep3': np.True_, 'Animal10_April16_gap_duration_10_Rep2': np.True_, 'Animal10_April16_offset_PPI_50_Rep2': np.True_, 'Animal10_April16_gap_duration_

In [18]:
animal_hits = {}
animal_misses = {}

for key, detected in signal_results.items():
    animal = key.split('_')[0]
    if detected:
        animal_hits[animal] = animal_hits.get(animal, 0) + 1
    else:
        animal_misses[animal] = animal_misses.get(animal, 0) + 1

animal_hit_rate = {}
for animal in animal_hits.keys() | animal_misses.keys():
    hits = animal_hits.get(animal, 0)
    misses = animal_misses.get(animal, 0)
    total = hits + misses
    animal_hit_rate[animal] = round(hits / total, 2) if total > 0 else None

if verbose: print(animal_hit_rate)

{'Animal1': 0.93, 'Animal15': 0.89, 'Animal4': 0.94, 'Animal10': 0.94, 'Animal2': 0.95, 'Animal6': 0.95}


In [19]:
""" discard_peakless = False
output = False """

if discard_peakless:
    excluded_counts = {}
    for k, df in adjusted_segments.items():
        if not signal_results.get(k, False):
            parts = k.split('_')
            experiment_parts = []
            for p in parts[2:]:
                if p.startswith("Rep"):
                    break
                experiment_parts.append(p)
            experiment = "_".join(experiment_parts)
            excluded_counts[experiment] = excluded_counts.get(experiment, 0) + 1

    if verbose:
        print("Excluded count per experiment (did not meet threshold):")
        for experiment, count in excluded_counts.items():
            print(f"{experiment}: {count}")

    # Discard entries without peaks
    data = {k: v for k, v in adjusted_segments.items() if signal_results.get(k, False)}
else:
    data = adjusted_segments
# ...existing code...

## Analysis

In [20]:
def exp_decay(t, A, tau, C):
    return A * np.exp(-t / tau) + C

main_reaction = {}

for entry in data:
    times = data[entry][0]
    values = data[entry][1]

    # Fixed window for all metrics except peakTime/peakValue
    time_window_mask = (times >= 0) & (times <= RMS_window)
    values_in_window = values[time_window_mask]
    times_in_window = times[time_window_mask]

    # Only consider peaks within 0-100ms after startle for peakTime and peakValue
    peak_window_mask = (times >= 0) & (times <= peak_window)
    values_peak_window = values[peak_window_mask]
    times_peak_window = times[peak_window_mask]

    indices_above_threshold = np.where((values > animal_thresholds[entry.split("_")[0]]) & (times > 0))[0]

    if len(indices_above_threshold) > 0:
        peak_start = times[indices_above_threshold[0]]

        # Find peak only in 0-100ms window
        if len(values_peak_window) > 0:
            max_index_in_peak_window = np.argmax(values_peak_window)
            max_value = values_peak_window.iloc[max_index_in_peak_window]
            max_time = times_peak_window.iloc[max_index_in_peak_window]
        else:
            max_value = np.nan
            max_time = np.nan

        buildup = max_time - peak_start if not np.isnan(max_time) else np.nan
        rms_value = np.sqrt(np.mean(np.square(values_in_window)))

        # --- POST-PEAK decay analysis (limit to 500 ms) ---
        decay_mask = (times > max_time) & (times <= max_time + decay_fit_window)
        times_decay = times[decay_mask] - max_time
        values_decay = values[decay_mask]

        tau = np.nan
        auc = np.nan
        peak_decay_tau = np.nan
        peak_decay_auc = np.nan

        if len(times_decay) > 5:
            try:
                p0 = [max_value, 100, np.min(values_decay)]
                params, _ = curve_fit(exp_decay, times_decay, values_decay, p0=p0, maxfev=10000)
                A, tau, C = params

                fitted_curve = exp_decay(times_decay, A, tau, C)
                auc = simpson(fitted_curve, x=times_decay)

                # --- PEAK DECAY analysis (limit to 500 ms) ---
                peak_mask = (values_decay > fitted_curve)
                if np.sum(peak_mask) > 5:
                    times_peak_decay = times_decay[peak_mask]
                    values_peak_decay = values_decay[peak_mask]
                    # Limit peak decay fit to max 500 ms after peak (already limited by times_decay)
                    peak_decay_limit_mask = times_peak_decay <= decay_fit_window
                    times_peak_decay = times_peak_decay[peak_decay_limit_mask]
                    values_peak_decay = values_peak_decay[peak_decay_limit_mask]
                    try:
                        p0_peak = [np.max(values_peak_decay), 100, np.min(values_peak_decay)]
                        params_peak, _ = curve_fit(exp_decay, times_peak_decay, values_peak_decay, p0=p0_peak, maxfev=10000)
                        A_peak, tau_peak, C_peak = params_peak
                        fitted_peak_curve = exp_decay(times_peak_decay, A_peak, tau_peak, C_peak)
                        peak_decay_auc = simpson(fitted_peak_curve, x=times_peak_decay)
                        peak_decay_tau = tau_peak
                    except Exception as e:
                        if verbose:
                            print(f"Peak decay fit failed for {entry}: {e}")
            except Exception as e:
                if verbose:
                    print(f"Decay fit failed for {entry}: {e}")

        # Save result
        main_reaction[entry] = {
            "reactionTime": [int(peak_start)],
            "peakTime": [int(max_time) if not np.isnan(max_time) else None],
            "difference": [int(buildup) if not np.isnan(buildup) else None],
            "peakValue": [int(max_value) if not np.isnan(max_value) else None],
            "RMS": [round(float(rms_value), 2)],
            "decayTau": [round(float(tau), 2)],
            "decayAUC": [round(float(auc), 2)],
            "peakDecayTau": [round(float(peak_decay_tau), 2)],
            "peakDecayAUC": [round(float(peak_decay_auc), 2)]
        }

    else:
        if discard_peakless:
            raise ValueError(f"No peak found for {entry}, something went really really wrong.")

if verbose: print(f"Created reaction data for {len(main_reaction)} segments.")
# ...existing code...

c:\Users\Galahad\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Galahad\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Galahad\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Galahad\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Galahad\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, 

Created reaction data for 1550 segments.


In [21]:
# peak to peak amplitude
for entry in data:
    times = data[entry][0]
    values = data[entry][1]

    # Peak-to-peak amplitude within 0-50ms after startle
    ptp_mask = (times >= 0) & (times <= PTPA_window)
    values_ptp_window = values[ptp_mask]
    if len(values_ptp_window) > 0:
        ptp_amplitude = float(np.max(values_ptp_window) - np.min(values_ptp_window))
    else:
        ptp_amplitude = None

    # ...existing analysis code...

    # After main_reaction[entry] is created, append peak-to-peak amplitude:
    if entry in main_reaction:
        main_reaction[entry]["PTPA"] = [ptp_amplitude]
# ...existing code...

In [22]:
# ...existing code...
for entry in data:
    times = data[entry][0]
    values = data[entry][1]

    # Find peak-to-trough amplitude within 0-200ms after startle
    asr_window_mask = (times >= 0) & (times <= PTTA_window)
    values_asr_window = values[asr_window_mask].reset_index(drop=True)  # Ensure integer indexing
    times_asr_window = times[asr_window_mask].reset_index(drop=True)

    asr_amplitude = None
    if len(values_asr_window) > 1:
        from scipy.signal import find_peaks

        peaks, _ = find_peaks(values_asr_window)
        troughs, _ = find_peaks(-values_asr_window)

        max_ptt = None
        for peak_idx in peaks:
            # Find all troughs after this peak
            troughs_after = troughs[troughs > peak_idx]
            if len(troughs_after) > 0:
                trough_idx = troughs_after[0]
                amplitude = values_asr_window.iloc[peak_idx] - values_asr_window.iloc[trough_idx]
                if (max_ptt is None) or (amplitude > max_ptt):
                    max_ptt = amplitude
        asr_amplitude = float(max_ptt) if max_ptt is not None else None

    # ...existing analysis code...

    # After main_reaction[entry] is created, append ASR amplitude:
    if entry in main_reaction:
        main_reaction[entry]["PTTA"] = [asr_amplitude]
# ...existing code...

In [23]:
top_10_reaction = {}
minus_top_10_reaction = {}

for key in top_10_keys:
    if key in main_reaction: top_10_reaction[key] = main_reaction[key]
for key in minus_top_10_keys:
    if key in main_reaction: minus_top_10_reaction[key] = main_reaction[key]

## Saving

In [24]:
# ...existing code...
def build_intermediate(reaction_dict):
    intermediate = defaultdict(lambda: {k: [] for k in [
        "animal", "sex", "date", "reactionTime", "peakTime", "difference", "peakValue", "PTPA", "PTTA", "RMS",
        "decayTau", "decayAUC", "peakDecayTau", "peakDecayAUC"
    ]})
    for entry, values in reaction_dict.items():
        parts = entry.split("_")
        animal, date = parts[0], parts[1]
        sex = sex_dict[animal.replace("Animal", "")]
        experiment = "_".join([p for p in parts[2:] if not p.lower().startswith("rep")])
        key = f"{animal}_{date}_{experiment}"
        intermediate[key]["animal"] = animal
        intermediate[key]["sex"] = sex
        intermediate[key]["date"] = date
        for k in intermediate[key]:
            if k in values:
                intermediate[key][k].extend(values[k] if isinstance(values[k], list) else [values[k]])
    return intermediate
# ...existing code...

intermediate_main = build_intermediate(main_reaction)
intermediate_top_10 = build_intermediate(top_10_reaction)
intermediate_minus_top_10 = build_intermediate(minus_top_10_reaction)

# ...existing code...

# Overview only for main
overview = [
    {
        "animal": key.split("_")[0],
        "sex": sex_dict[key.split("_")[0].replace("Animal", "")],
        "experiment": "_".join(key.split("_")[2:]),
        **{
            k: round(
                np.mean([
                    v for key2, metrics in intermediate_main.items()
                    if key2.split("_")[0] == key.split("_")[0]
                    and "_".join(key2.split("_")[2:]) == "_".join(key.split("_")[2:])
                    and sex_dict[key2.split("_")[0].replace("Animal", "")] == sex_dict[key.split("_")[0].replace("Animal", "")]
                    for v in metrics[k] if v is not None
                ]), 2
            ) if any(v is not None for key2, metrics in intermediate_main.items()
                    if key2.split("_")[0] == key.split("_")[0]
                    and "_".join(key2.split("_")[2:]) == "_".join(key.split("_")[2:])
                    and sex_dict[key2.split("_")[0].replace("Animal", "")] == sex_dict[key.split("_")[0].replace("Animal", "")]
                    for v in metrics[k]) else np.nan
            for k in metrics if k not in ["animal", "sex", "date"]
        }
    }
    for key, metrics in intermediate_main.items()
    if key.split("_")[1] == min([
        k.split("_")[1] for k in intermediate_main
        if k.split("_")[0] == key.split("_")[0] and "_".join(k.split("_")[2:]) == "_".join(key.split("_")[2:])
    ])
]

# ...existing code...

# Results for each dataset
def build_results(intermediate):
    return [
        {
            "animal": key.split("_")[0],
            "sex": sex_dict[key.split("_")[0].replace("Animal", "")],
            "date": key.split("_")[1],
            "experiment": "_".join(key.split("_")[2:]),
            **{k: metrics[k] for k in metrics if k not in ["animal", "sex", "date"]}
        }
        for key, metrics in intermediate.items()
    ]

results = build_results(intermediate_main)
results_top_10 = build_results(intermediate_top_10)
results_minus_top_10 = build_results(intermediate_minus_top_10)
# ...existing code...

In [40]:
# ...existing code...
def _pgf_escape(s):
    """Escape minimal set of characters that break LaTeX/PGF output."""
    if s is None:
        return s
    repl = {
        "\\": r"\textbackslash{}",
        "_": r"\_",
        "%": r"\%",
        "&": r"\&",
        "#": r"\#",
        "{": r"\{",
        "}": r"\}",
        "$": r"\$",
    }
    for k, v in repl.items():
        s = s.replace(k, v)
    return s

def plot_segment(name, segments, reaction, save_dir=None):
    if name not in segments or name not in reaction:
        print(f"{name} not found in segments or reaction data.")
        return

    import matplotlib as mpl
    mpl.rcParams.update({
        "pgf.texsystem": "pdflatex",
        "font.family": "serif",
        "text.usetex": False,   # avoid calling external LaTeX
        "pgf.rcfonts": False,
    })

    df = segments[name]
    times = df[0]
    values = df[1]
    startle_x = 0

    animal = name.split("_")[0]
    plot_noise = animal_noise.get(animal)
    plot_threshold_1 = plot_noise * detection_threshold 
    plot_threshold_2 = animal_thresholds[animal]

    # Extract analysis results
    metrics = reaction[name]
    peak_time = metrics.get("peakTime", [None])[0]
    peak_value = metrics.get("peakValue", [None])[0]
    decay_tau = metrics.get("decayTau", [None])[0] 
    decay_auc = metrics.get("decayAUC", [None])[0]
    peak_decay_tau = metrics.get("peakDecayTau", [None])[0]
    peak_decay_auc = metrics.get("peakDecayAUC", [None])[0]
    ptpa = metrics.get("PTPA", [None])[0]
    ptta = metrics.get("PTTA", [None])[0]

    # Get experiment name for prepulse lookup
    parts = name.split("_")
    experiment = "_".join([p for p in parts[2:] if not p.lower().startswith("rep")])
    prepulse_time = prepulse_times.get(experiment, None)

    fig, ax = plt.subplots(figsize=(8, 6))

    # rasterize dense signal lines so PGF stays manageable
    ax.plot(times, values, color="#000000", linewidth=1, label="Signal", rasterized=True)

    # --- Threshold & timing markers (vector) ---
    ax.axvline(x=startle_x, color="#FF0000", linewidth=2, linestyle='dotted', label="Startle")
    ax.axvline(x=RMS_window, color="#8F7F37", linewidth=2, linestyle='dotted',
               label=f"RMS window (t={RMS_window}ms)")
    ax.axvline(x=PTPA_window, color="#419741", linewidth=2, linestyle='dotted',
               label=f"PTPA window (t={PTPA_window}ms)")
    ax.axvline(x=PTTA_window, color="#612670", linewidth=2, linestyle='dotted',
               label=f"PTTA window (t={PTTA_window}ms)")

    ax.axhline(y=plot_threshold_1, color="#CE1195", linewidth=2, linestyle='dotted',
               label=f"Detection Threshold (k={detection_threshold})")
    ax.axhline(y=plot_noise, color="#6F52BF", linewidth=2, linestyle='dotted',
               label=f"Background Noise ({plot_noise:.2f})")

    if prepulse_time is not None:
        rel_prepulse_time = int(prepulse_time - startle_times.get(experiment, 0))
        ax.axvline(x=rel_prepulse_time, color="#6F52BF", linewidth=2, linestyle='dotted',
                   label=f"Prepulse ({rel_prepulse_time}ms)")

    # --- Mark peak ---
    if peak_time is not None and peak_value is not None:
        ax.plot(peak_time, peak_value, 'ro', label=_pgf_escape(f"Peak ({peak_time}ms, {peak_value})"))
        ax.axvline(x=peak_time, color="#AA0000", linewidth=1.5, linestyle='--')

    # --- Text annotations ---
    if ptpa is not None:
        ax.text(0.98, 0.92, f"PTPA: {ptpa:.2f}", transform=ax.transAxes,
                fontsize=12, color="#0072B2", ha='right', va='center', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
    if ptta is not None:
        ax.text(0.98, 0.86, f"PTTA: {ptta:.2f}", transform=ax.transAxes,
                fontsize=12, color="#D55E00", ha='right', va='center', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

    # --- Plot fitted decay curve (rasterize fit if dense) ---
    if decay_tau is not None and not np.isnan(decay_tau) and peak_time is not None and peak_value is not None:
        try:
            decay_mask = times > peak_time
            decay_times = times[decay_mask]
            decay_values = values[decay_mask]
            shifted_times = decay_times - peak_time

            # Estimate fit again with starting params
            p0 = [peak_value, decay_tau, np.min(decay_values)]
            params, _ = curve_fit(exp_decay, shifted_times, decay_values, p0=p0, maxfev=10000)
            A, tau, C = params

            fitted = exp_decay(shifted_times, A, tau, C)
            # Use ASCII label (avoid Unicode/math) then escape
            decay_label = f"Decay Fit (tau={tau:.2f} ms, AUC={decay_auc:.2f})"
            ax.plot(decay_times, fitted, 'r--', label=_pgf_escape(decay_label), rasterized=True)

        except Exception as e:
            print(f"Decay fitting failed for {name}: {e}")

    ax.set_xlabel("Time (ms)", fontsize=12)
    ax.set_ylabel("ValueG", fontsize=12)
    ax.set_title(_pgf_escape(name), fontsize=14)
    ax.grid(True, linestyle="--", alpha=0.7)
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))  # <-- Legend outside the plot

    # Save figure as PGF to export_folder only
    os.makedirs(export_folder, exist_ok=True)
    pgf_path = os.path.join(export_folder, f"{name}.pgf")
    try:
        fig.savefig(pgf_path, format="pgf", bbox_inches='tight')
        print(f"Saved PGF to {pgf_path}")
    except Exception as e:
        print(f"PGF save failed for {name}: {e}")

    plt.close(fig)
# ...existing code...

In [41]:
example_name = list(data.keys())[example_number] # get example segment
plot_segment(example_name, data, main_reaction) # plot example segment

Saved PGF to C:\Users\Galahad\Desktop\Thesis\images\Animal10_April16_offset_PPI_8_Rep2.pgf


In [27]:
""" # Save the example segment as a CSV file

example_name = list(data.keys())[example_number]
example_df = data[example_name]

# If example_df is a DataFrame, save directly; if not, convert to DataFrame
if isinstance(example_df, pd.DataFrame):
    df_to_save = example_df
else:
    df_to_save = pd.DataFrame(example_df)

output_example_path = os.path.join(output_dir, f"{example_name}_segment.csv")
df_to_save.to_csv(output_example_path, index=False)

print(f"Saved example segment to: {output_example_path}")

# Save the example segment as a CSV file

example_name = list(data.keys())[example_number+22]
example_df = data[example_name]

# If example_df is a DataFrame, save directly; if not, convert to DataFrame
if isinstance(example_df, pd.DataFrame):
    df_to_save = example_df
else:
    df_to_save = pd.DataFrame(example_df)

output_example_path = os.path.join(output_dir, f"{example_name}_segment.csv")
df_to_save.to_csv(output_example_path, index=False)

print(f"Saved example segment to: {output_example_path}")

# Save the example segment as a CSV file

example_name = list(data.keys())[example_number-4]
example_df = data[example_name]

# If example_df is a DataFrame, save directly; if not, convert to DataFrame
if isinstance(example_df, pd.DataFrame):
    df_to_save = example_df
else:
    df_to_save = pd.DataFrame(example_df)

output_example_path = os.path.join(output_dir, f"{example_name}_segment.csv")
df_to_save.to_csv(output_example_path, index=False)

print(f"Saved example segment to: {output_example_path}") """

' # Save the example segment as a CSV file\n\nexample_name = list(data.keys())[example_number]\nexample_df = data[example_name]\n\n# If example_df is a DataFrame, save directly; if not, convert to DataFrame\nif isinstance(example_df, pd.DataFrame):\n    df_to_save = example_df\nelse:\n    df_to_save = pd.DataFrame(example_df)\n\noutput_example_path = os.path.join(output_dir, f"{example_name}_segment.csv")\ndf_to_save.to_csv(output_example_path, index=False)\n\nprint(f"Saved example segment to: {output_example_path}")\n\n# Save the example segment as a CSV file\n\nexample_name = list(data.keys())[example_number+22]\nexample_df = data[example_name]\n\n# If example_df is a DataFrame, save directly; if not, convert to DataFrame\nif isinstance(example_df, pd.DataFrame):\n    df_to_save = example_df\nelse:\n    df_to_save = pd.DataFrame(example_df)\n\noutput_example_path = os.path.join(output_dir, f"{example_name}_segment.csv")\ndf_to_save.to_csv(output_example_path, index=False)\n\nprint(f"

In [42]:
# Plot all offset_ASR experiments

offset_asr_keys = [k for k in data if "_offset_ASR_" in k]
print(f"Found {len(offset_asr_keys)} offset_ASR segments.")

for key in offset_asr_keys:
    plot_segment(key, data, main_reaction)

Found 90 offset_ASR segments.
Animal10_April16_offset_ASR_Rep1 not found in segments or reaction data.
Animal10_April16_offset_ASR_Rep2 not found in segments or reaction data.
Animal10_April16_offset_ASR_Rep3 not found in segments or reaction data.
Animal10_April16_offset_ASR_Rep4 not found in segments or reaction data.
Animal10_April16_offset_ASR_Rep5 not found in segments or reaction data.
Animal10_June26_offset_ASR_Rep1 not found in segments or reaction data.
Animal10_June26_offset_ASR_Rep2 not found in segments or reaction data.
Animal10_June26_offset_ASR_Rep3 not found in segments or reaction data.
Animal10_June26_offset_ASR_Rep4 not found in segments or reaction data.
Animal10_June26_offset_ASR_Rep5 not found in segments or reaction data.
Animal10_May20_offset_ASR_Rep1 not found in segments or reaction data.
Animal10_May20_offset_ASR_Rep2 not found in segments or reaction data.
Animal10_May20_offset_ASR_Rep3 not found in segments or reaction data.
Animal10_May20_offset_ASR_Rep4 n

In [43]:
# Plot only Animal10_June26_offset_ASR_Rep4 with one legend

segment_key = "Animal10_June26_offset_ASR_Rep4"

if segment_key in data and segment_key in main_reaction:
    print(f"Plotting segment: {segment_key}")
    plot_segment(segment_key, data, main_reaction)
else:
    print(f"{segment_key} not found in data or main_reaction.")

Animal10_June26_offset_ASR_Rep4 not found in data or main_reaction.


In [44]:
import matplotlib.pyplot as plt

# Define the segment keys
example_name = list(data.keys())[example_number]
segment_key = "Animal10_June26_offset_ASR_Rep4"

# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot example_name on the left
if example_name in data and example_name in main_reaction:
    plt.sca(axes[0])
    plot_segment(example_name, data, main_reaction)
    axes[0].set_title(f"Example: {example_name}")
else:
    print(f"{example_name} not found in data or main_reaction.")

# Plot segment_key on the right
if segment_key in data and segment_key in main_reaction:
    plt.sca(axes[1])
    plot_segment(segment_key, data, main_reaction)
    axes[1].set_title(f"Offset ASR: {segment_key}")
else:
    print(f"{segment_key} not found in data or main_reaction.")

plt.tight_layout()
plt.show()

Saved PGF to C:\Users\Galahad\Desktop\Thesis\images\Animal10_April16_offset_PPI_8_Rep2.pgf
Animal10_June26_offset_ASR_Rep4 not found in data or main_reaction.


In [31]:
def get_number(s):
    match = re.search(r'(\d+)$', s)  
    return int(match.group(1)) if match else None

def get_experiment_type(s):
    return re.sub(r'[_\s-]?\d+$', '', s)

def get_animal_number(s):
    match = re.search(r'(\d+)', s)
    return int(match.group(1)) if match else None

In [32]:
# Exclude a set of segment names from all downstream analysis
EXCLUDE_SEGMENTS = {
    # Add the full segment names you want to exclude, e.g.:
    "Animal4_May20_offset_PPI_50_Rep3"
    # "Animal10_June26_offset_ASR_Rep4",
}

# Remove excluded segments from data and main_reaction
data = {k: v for k, v in data.items() if k not in EXCLUDE_SEGMENTS}
main_reaction = {k: v for k, v in main_reaction.items() if k not in EXCLUDE_SEGMENTS}

In [33]:
if output:
    os.makedirs(output_dir, exist_ok=True)

    # Always use (and overwrite) a single output folder
    subfolder_path = output_dir

    os.makedirs(subfolder_path, exist_ok=True)

    dfs = [pd.DataFrame(overview), pd.DataFrame(results), pd.DataFrame(results_top_10), pd.DataFrame(results_minus_top_10)]

    for i, df in enumerate(dfs):
        # Exclude rows where tau (peakDecayTau) is negative before saving
        if 'peakDecayTau' in df.columns:
            df = df[df['peakDecayTau'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x) >= 0 if 'peakDecayTau' in df.columns else True].copy()
        elif 'tau' in df.columns:
            df = df[df['tau'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x) >= 0 if 'tau' in df.columns else True].copy()

        # Sorting (use .loc to avoid SettingWithCopyWarning)
        df.loc[:, 'has_number'] = df['experiment'].str.contains(r'\d+$')
        df.loc[:, 'experiment_number'] = df['experiment'].apply(get_number)
        df.loc[:, 'experiment_type'] = df['experiment'].apply(get_experiment_type)
        df.loc[:, 'animal_number'] = df['animal'].apply(get_animal_number)
        try:
            df = df.sort_values(['date', 'animal_number', 'has_number', 'experiment_type', 'experiment_number'])
        except KeyError:
            df = df.sort_values(['animal_number', 'has_number', 'experiment_type', 'experiment_number'])

        df = df.drop(columns=['has_number', 'experiment_type', 'experiment_number', 'animal_number'])

        if not include_decay_fit:
            df = df.rename(columns={'decayAUC': 'AUC'}) 

        df = df.drop(columns=['reactionTime', 'difference', 'peakDecayTau', 'peakDecayAUC', 'decayTau'])
        df = df.rename(columns={'peakTime': 'reactionTime'})

        df = df.reset_index(drop=True)

        # Naming and saving
        filename = filenames[i]
        output_path = os.path.join(subfolder_path, f"{filename}.xlsx")

        df.to_excel(output_path, index=False)

    if verbose: print(f"Saved results to: {subfolder_path}")

In [34]:
# Plot all segments where tau or peakDecayTau is negative
for key, metrics in main_reaction.items():
    tau = metrics.get("tau", [None])[0]
    peak_tau = metrics.get("peakDecayTau", [None])[0]
    if (tau is not None and tau < 0) or (peak_tau is not None and peak_tau < 0):
        print(f"Plotting segment with negative tau: {key} (tau={tau}, peakDecayTau={peak_tau})")
        plot_segment(key, data, main_reaction)